In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (roc_auc_score, accuracy_score, confusion_matrix, classification_report, precision_recall_curve, recall_score)
from sklearn.utils import resample
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Dropout, Activation
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

In [2]:
data = pd.read_csv('Breast_cancer_data.csv')
X = data.drop('diagnosis', axis=1).values
y = data['diagnosis'].values
X_train, X_test, y_train, y_test = train_test_split(
    X, y, stratify=y, test_size=0.2, random_state=42
)
X_train, X_val, y_train, y_val = train_test_split(
    X_train, y_train, stratify=y_train, test_size=0.25, random_state=42
)

In [3]:
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

In [4]:
# Combine for resampling
df = pd.DataFrame(X_train)
df['label'] = y_train
df_majority = df[df.label == 0]
df_minority = df[df.label == 1]
# Upsample malignant to 3x majority
df_min_up = resample(df_minority, replace=True, n_samples=len(df_majority)*3, random_state=42)
df_resampled = pd.concat([df_majority, df_min_up])
X_train_up = df_resampled.drop('label', axis=1).values
y_train_up = df_resampled['label'].values
print('Resampled distribution:', pd.Series(y_train_up).value_counts())

Resampled distribution: 1    381
0    127
Name: count, dtype: int64


In [5]:
def focal_loss(alpha=0.25, gamma=2.0):
    def loss(y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1-1e-7)
        cross_entropy = -y_true * tf.math.log(y_pred) - (1-y_true) * tf.math.log(1-y_pred)
        weight = alpha * y_true * tf.math.pow(1-y_pred, gamma) + (1-alpha)*(1-y_true)*tf.math.pow(y_pred, gamma)
        return tf.reduce_mean(weight * cross_entropy)
    return loss

In [6]:
def build_model(input_dim):
    model = Sequential([
        Dense(128, input_shape=(input_dim,)), BatchNormalization(), Activation('relu'), Dropout(0.4),
        Dense(64), BatchNormalization(), Activation('relu'), Dropout(0.3),
        Dense(32), Activation('relu'), Dropout(0.2),
        Dense(1, activation='sigmoid')
    ])
    model.compile(
        loss=focal_loss(alpha=0.3, gamma=2.0),
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3),
        metrics=['accuracy']
    )
    return model
model = build_model(X_train_up.shape[1])
model.summary()

C:\Users\sebas\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense (Dense)                   │ (None, 128)            │           768 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 128)            │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation (Activation)         │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 128)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         8,256 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_1 (Activation)       │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ activation_2 (Activation)       │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,905 (46.50 KB)

 Trainable params: 11,521 (45.00 KB)

 Non-trainable params: 384 (1.50 KB)

In [7]:
callbacks = [
    EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True),
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6)
]
history = model.fit(
    X_train_up, y_train_up,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=32,
    callbacks=callbacks,
    verbose=1
)

Epoch 1/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 3s 24ms/step - accuracy: 0.7128 - loss: 0.0765 - val_accuracy: 0.9474 - val_loss: 0.0491 - learning_rate: 0.0010
Epoch 2/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8539 - loss: 0.0342 - val_accuracy: 0.9386 - val_loss: 0.0401 - learning_rate: 0.0010
Epoch 3/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step - accuracy: 0.8527 - loss: 0.0382 - val_accuracy: 0.9386 - val_loss: 0.0356 - learning_rate: 0.0010
Epoch 4/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8997 - loss: 0.0342 - val_accuracy: 0.9386 - val_loss: 0.0331 - learning_rate: 0.0010
Epoch 5/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.8607 - loss: 0.0289 - val_accuracy: 0.9298 - val_loss: 0.0307 - learning_rate: 0.0010
Epoch 6/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.9207 - loss: 0.0229 - val_accuracy: 0.9298 - val_loss: 0.0290 - learning_rate: 0.0010
Epoch 7/100
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.9222 - loss: 0.0253 - val_a

In [8]:
y_val_prob = model.predict(X_val).ravel()
prec, rec, thr = precision_recall_curve(y_val, y_val_prob)
idx = np.where(rec >= 1.0)[0]
best_thresh = thr[idx[-1]] if len(idx) else 0.5
print(f'Optimal Threshold: {best_thresh:.3f}')

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
Optimal Threshold: 0.324


In [9]:
y_test_prob = model.predict(X_test).ravel()
y_test_pred = (y_test_prob >= best_thresh).astype(int)
print('Recall on positive cancer test:', recall_score(y_test, y_test_pred))
print('Confusion Matrix:\n', confusion_matrix(y_test, y_test_pred))
print(classification_report(y_test, y_test_pred))

4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step 
Recall on positive cancer test: 0.9583333333333334
Confusion Matrix:
 [[36  6]
 [ 3 69]]
              precision    recall  f1-score   support

           0       0.92      0.86      0.89        42
           1       0.92      0.96      0.94        72

    accuracy                           0.92       114
   macro avg       0.92      0.91      0.91       114
weighted avg       0.92      0.92      0.92       114

